In [1]:
import os
import spacy
import networkx as nx
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

try:
    nlp = spacy.load("en_core_web_sm")
    print("SpaCy 'en_core_web_sm' 모델이 성공적으로 로드되었습니다.")
except OSError:
    print("SpaCy 'en_core_web_sm' 모델을 다운로드합니다...")
    os.system("python -m spacy download en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")
    print("SpaCy 'en_core_web_sm' 모델 다운로드 및 로드 완료.")

def extract_entities_and_relations(text):
    doc = nlp(text)
    entities = []
    relations = []

    for chunk in doc.noun_chunks:
        entities.append(chunk.text.lower())

    for i in range(len(doc) - 1):
        token1 = doc[i]
        token2 = doc[i+1]

        if token1.pos_ == "NOUN" and token2.pos_ == "NOUN":
            relations.append((token1.lemma_.lower(), "has_part" if token2.dep_ == "compound" else "related_to", token2.lemma_.lower()))
        elif token1.ent_type_ and token2.ent_type_:
            relations.append((token1.text.lower(), "co_occurs_with", token2.text.lower()))
        elif token1.pos_ == "NOUN" and token2.dep_ == "prep" and i + 2 < len(doc) and doc[i+2].pos_ == "NOUN":
            relations.append((token1.lemma_.lower(), "is_" + token2.text.lower() + "_of", doc[i+2].lemma_.lower()))
            
    extracted_nouns = []
    for token in doc:
        if token.pos_ == "NOUN":
            extracted_nouns.append(token.lemma_.lower())

    for i in range(len(extracted_nouns)):
        for j in range(i + 1, len(extracted_nouns)):
            if extracted_nouns[i] != extracted_nouns[j]:
                relations.append((extracted_nouns[i], "co_occurs_in_document", extracted_nouns[j]))

    entities = list(set(entities + extracted_nouns))
    relations = list(set(relations))

    return entities, relations

def clean_markdown(md_text):
    text = re.sub(r'^#+\s*.*$', '', md_text, flags=re.MULTILINE)
    text = re.sub(r'(\*\*|__)(.*?)\1', r'\2', text)
    text = re.sub(r'(\*|_)(.*?)\1', r'\2', text)
    text = re.sub(r'\[(.*?)\]\(.*?\)', r'\1', text)
    text = re.sub(r'^\s*[-*+]\s+', '', text, flags=re.MULTILINE)
    text = re.sub(r'^-{3,}\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\*{3,}\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'```.*?```', '', text, flags=re.DOTALL)
    text = re.sub(r'`(.*?)`', r'\1', text)
    text = re.sub(r'\n\n+', '\n', text)
    return text.strip()

def build_knowledge_graph(data_folder_path):
    G = nx.DiGraph()
    print(f"'{data_folder_path}' 폴더에서 Markdown 파일들을 읽고 지식 그래프를 구축합니다...")
    
    markdown_files = [f for f in os.listdir(data_folder_path) if f.endswith(".md")]
    for filename in tqdm(markdown_files, desc="지식 그래프 구축"):
        filepath = os.path.join(data_folder_path, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            md_text = f.read()
            plain_text = clean_markdown(md_text)
            entities, relations = extract_entities_and_relations(plain_text)

            for entity in entities:
                G.add_node(entity)

            for subj, pred, obj in relations:
                if subj in G.nodes and obj in G.nodes:
                    G.add_edge(subj, obj, relation=pred)
                else:
                    if subj not in G.nodes:
                        G.add_node(subj)
                    if obj not in G.nodes:
                        G.add_node(obj)
                    G.add_edge(subj, obj, relation=pred)
    
    print(f"지식 그래프 구축 완료. 총 {len(markdown_files)}개의 파일을 처리했습니다.")
    print(f"지식 그래프에는 현재 {G.number_of_nodes()}개의 노드와 {G.number_of_edges()}개의 엣지가 있습니다.")
    return G

def visualize_graph(graph, title="Knowledge Graph"):
    print("지식 그래프 시각화를 시작합니다...")
    plt.figure(figsize=(15, 10))
    pos = nx.spring_layout(graph, k=0.15, iterations=20) 

    nx.draw_networkx_nodes(graph, pos, node_size=2000, node_color="skyblue", alpha=0.9)
    nx.draw_networkx_labels(graph, pos, font_size=8, font_weight="bold")
    
    nx.draw_networkx_edges(graph, pos, edge_color="gray", arrows=True, alpha=0.6)
    edge_labels = nx.get_edge_attributes(graph, 'relation')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_size=7, verticalalignment='baseline')

    plt.title(title)
    plt.axis('off')
    plt.show()
    print("지식 그래프 시각화가 완료되었습니다.")

# --- 간단한 시각화를 위한 추가 함수 ---

def get_subgraph_around_keywords(full_graph, keywords, depth=1):
    sub_nodes = set()
    for keyword in keywords:
        if keyword in full_graph:
            sub_nodes.add(keyword)
            # 특정 깊이까지 이웃 노드 추가
            for neighbor in nx.ego_graph(full_graph, keyword, radius=depth).nodes():
                sub_nodes.add(neighbor)
    
    # 선택된 노드로 하위 그래프 생성
    subgraph = full_graph.subgraph(sub_nodes)
    return subgraph

if __name__ == "__main__":
    data_folder = "./data/split_file/anatomy"

    print("데이터 폴더 존재 여부를 확인합니다...")
    if not os.path.exists(data_folder):
        print(f"데이터 폴더 '{data_folder}'가 존재하지 않습니다. 폴더를 생성합니다...")
        os.makedirs(data_folder)
        
        dummy_files = [
            "1_Embryology.md", "2_Osteology.md", "3_Syndesmology.md",
            "4_Myology.md", "5_Angiology.md", "6_The_Arteries.md",
            "7_The_Veins.md", "8_The_Lymphatic_System.md", "9_Neurology.md",
            "10_The_Organs_of_the_Senses_and_the_Common_Integument.md",
            "11_Splanchnology.md", "12_Surface_Anatomy_and_Surface_Markings.md"
        ]

        file_contents = {
            "1_Embryology.md": "Embryology is the study of embryos. The human embryo develops after fertilization.",
            "2_Osteology.md": "Osteology is the study of bones. The femur is a long bone in the thigh. Muscles attach to bones.",
            "3_Syndesmology.md": "Syndesmology is the study of joints and ligaments. Ligaments connect bones to other bones.",
            "4_Myology.md": "Myology is the study of muscles. Muscles enable movement. There are different types of muscle tissue.",
            "5_Angiology.md": "Angiology is the study of the circulatory and lymphatic systems. It includes arteries, veins, and lymphatic vessels.",
            "6_The_Arteries.md": "Arteries carry oxygenated blood away from the heart. The aorta is the largest artery.",
            "7_The_Veins.md": "Veins carry deoxygenated blood back to the heart. Valves in veins prevent backflow.",
            "8_The_Lymphatic_System.md": "The lymphatic system is part of the immune system. It includes lymph nodes and lymphatic vessels.",
            "9_Neurology.md": "Neurology is the study of the nervous system. The brain and spinal cord are central nervous system organs.",
            "10_The_Organs_of_the_Senses_and_the_Common_Integument.md": "This section covers sensory organs like eyes and ears, and the integumentary system, which includes skin, hair, and nails.",
            "11_Splanchnology.md": "Splanchnology is the study of internal organs or viscera. It includes organs of the digestive, respiratory, and urogenital systems.",
            "12_Surface_Anatomy_and_Surface_Markings.md": "Surface anatomy studies external features of the body. Surface markings indicate underlying structures."
        }

        print("더미 파일을 생성합니다...")
        for filename in tqdm(dummy_files, desc="더미 파일 생성"):
            filepath = os.path.join(data_folder, filename)
            content = file_contents.get(filename, f"This is a dummy file for {filename.replace('.md', '').replace('_', ' ')}.")
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(content)
        print("모든 더미 파일 생성이 완료되었습니다.")
    else:
        print(f"데이터 폴더 '{data_folder}'가 이미 존재합니다. 기존 파일을 사용합니다.")
    
    full_knowledge_graph = build_knowledge_graph(data_folder)

    # --- 간단한 시각화를 위한 코드 추가 ---
    print("\n간단한 시각화를 위해 특정 키워드 주변의 하위 그래프를 생성합니다.")
    
    # 시각화하고 싶은 키워드를 여기에 입력하세요 (소문자로 입력)
    keywords_to_visualize = ["brain", "heart", "muscle", "bone", "artery"] 
    
    # 키워드 주변의 하위 그래프 생성 (깊이 1로 설정하여 너무 커지지 않도록 함)
    sub_graph = get_subgraph_around_keywords(full_knowledge_graph, keywords_to_visualize, depth=1)

    print(f"시각화할 하위 그래프: 노드 수 {sub_graph.number_of_nodes()}, 엣지 수 {sub_graph.number_of_edges()}")
    
    if sub_graph.number_of_nodes() > 0:
        visualize_graph(sub_graph, title="Selected Anatomy Knowledge Subgraph")
    else:
        print("선택된 키워드와 관련된 노드가 없어 그래프를 시각화할 수 없습니다.")

SpaCy 'en_core_web_sm' 모델이 성공적으로 로드되었습니다.
데이터 폴더 존재 여부를 확인합니다...
데이터 폴더 './data/split_file/anatomy'가 이미 존재합니다. 기존 파일을 사용합니다.
'./data/split_file/anatomy' 폴더에서 Markdown 파일들을 읽고 지식 그래프를 구축합니다...


지식 그래프 구축: 100%|██████████| 12/12 [04:44<00:00, 23.68s/it]


지식 그래프 구축 완료. 총 12개의 파일을 처리했습니다.
지식 그래프에는 현재 50656개의 노드와 8567713개의 엣지가 있습니다.

간단한 시각화를 위해 특정 키워드 주변의 하위 그래프를 생성합니다.
시각화할 하위 그래프: 노드 수 4834, 엣지 수 8480058
지식 그래프 시각화를 시작합니다...


: 